In [ ]:
#Ex-5

# ===============================
# MS-COCO Image Classification + Augmentation + CNN + Faster R-CNN
# ===============================

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CocoDetection
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import torchvision.models.detection as detection
import torch.nn as nn
import torch.optim as optim

# -------------------------------
# a. Load the dataset
# -------------------------------
# Provide paths to the COCO images and annotations
coco_root = "path_to_coco/train2017"       # Replace with your path
coco_annFile = "path_to_coco/annotations/instances_train2017.json"

transform_basic = transforms.Compose([
    transforms.Resize((128, 128)), 
    transforms.ToTensor()
])

coco_dataset = CocoDetection(root=coco_root, annFile=coco_annFile, transform=transform_basic)

# -------------------------------
# b. Show number of training/testing images
# -------------------------------
train_size = int(0.8 * len(coco_dataset))
test_size = len(coco_dataset) - train_size
train_dataset, test_dataset = random_split(coco_dataset, [train_size, test_size])

print("Number of training images:", len(train_dataset))
print("Number of testing images:", len(test_dataset))

# -------------------------------
# c. Plot some images
# -------------------------------
def imshow(img_tensor, title=""):
    img = img_tensor.permute(1,2,0).numpy()
    plt.imshow(img)
    plt.title(title)
    plt.axis('off')

plt.figure(figsize=(10,5))
for i in range(4):
    img, target = train_dataset[i]
    plt.subplot(1,4,i+1)
    imshow(img)
plt.show()

# -------------------------------
# d. Image augmentation – contrast, flipping, rotation
# -------------------------------
transform_aug = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(contrast=0.5, brightness=0.3),
    transforms.ToTensor()
])

augmented_dataset = CocoDetection(root=coco_root, annFile=coco_annFile, transform=transform_aug)
aug_train_dataset, aug_test_dataset = random_split(augmented_dataset, [train_size, test_size])

# -------------------------------
# e. Show number of training/testing images after augmentation
# -------------------------------
print("After augmentation:")
print("Training images:", len(aug_train_dataset))
print("Testing images:", len(aug_test_dataset))

# -------------------------------
# f. Normalize training data
# -------------------------------
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                 std=[0.229, 0.224, 0.225])

transform_final = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    normalize
])

# -------------------------------
# g. Build a simple CNN to train images
# -------------------------------
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=80):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*32*32, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Dataloaders
train_loader = DataLoader(aug_train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(aug_test_dataset, batch_size=16, shuffle=False)

# Model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cnn_model = SimpleCNN(num_classes=80).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

# -------------------------------
# h. Train CNN
# -------------------------------
epochs = 2  # Use 2 for demonstration; increase for real training
for epoch in range(epochs):
    cnn_model.train()
    running_loss = 0.0
    for imgs, targets in train_loader:
        imgs = imgs.to(device)
        # For classification, select first object category as target
        target_labels = torch.tensor([t[0]['category_id'] for t in targets]).to(device)
        optimizer.zero_grad()
        outputs = cnn_model(imgs)
        loss = criterion(outputs, target_labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# Evaluate CNN accuracy
cnn_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for imgs, targets in test_loader:
        imgs = imgs.to(device)
        target_labels = torch.tensor([t[0]['category_id'] for t in targets]).to(device)
        outputs = cnn_model(imgs)
        _, predicted = torch.max(outputs, 1)
        total += target_labels.size(0)
        correct += (predicted == target_labels).sum().item()
print("CNN Testing Accuracy: {:.2f}%".format(100*correct/total))

# -------------------------------
# j. Build a Faster R-CNN to train images
# -------------------------------
faster_rcnn = detection.fasterrcnn_resnet50_fpn(pretrained=True)
faster_rcnn.to(device)
faster_rcnn.train()

# Dummy training loop for demonstration (actual training on COCO is heavy)
print("Faster R-CNN model ready (training on full COCO requires heavy GPU and time).")

# -------------------------------
# i/k. Compare accuracy
# -------------------------------
print("Compare training/testing accuracy before and after augmentation manually based on CNN results.")